In [ ]:
# Emergency fix implementation
# This cell shows the exact code changes needed in LexicalEditor.tsx

fix_lines_58_to_77 = """
// EMERGENCY FIX: Comment out HR_TRANSFORMER to prevent system freeze

// Horizontal Rule transformer (missing from default TRANSFORMERS)
/* DISABLED DUE TO RACE CONDITION BUG
const HR_TRANSFORMER: ElementTransformer = {
  dependencies: [HorizontalRuleNode],
  export: (node) => {
    return $isHorizontalRuleNode(node) ? '***' : null;
  },
  regExp: /^(---|\*\*\*|___)\s?$/,
  replace: (parentNode, _1, _2, isImport) => {
    const line = $createHorizontalRuleNode();
    if (isImport || parentNode.getNextSibling() != null) {
      parentNode.replace(line);
    } else {
      parentNode.insertBefore(line);
    }
    line.selectNext();
  },
  type: 'element',
};
*/

// Combined transformers including HR
// const EXTENDED_TRANSFORMERS = [HR_TRANSFORMER, ...TRANSFORMERS];
const EXTENDED_TRANSFORMERS = TRANSFORMERS; // EMERGENCY FIX: Use default only
"""

print("Apply this fix to src/sidebar/components/LexicalEditor.tsx lines 58-77:")
print(fix_lines_58_to_77)

### Option 3: Fix Custom Implementation (HIGHER RISK)
```typescript
// If keeping custom HR_TRANSFORMER, apply these fixes:

// 1. Remove conflicting patterns - use ONLY dashes
const HR_TRANSFORMER: ElementTransformer = {
  dependencies: [HorizontalRuleNode],
  export: (node) => {
    return $isHorizontalRuleNode(node) ? '---' : null; // Changed from '***'
  },
  regExp: /^---\s?$/, // ONLY dashes, no *** or ___
  replace: (parentNode, _1, _2, isImport) => {
    // Add safety check
    if ($isCodeNode(parentNode)) {
      return false; // Don't process inside code blocks
    }
    
    const line = $createHorizontalRuleNode();
    if (isImport || parentNode.getNextSibling() != null) {
      parentNode.replace(line);
    } else {
      parentNode.insertBefore(line);
    }
    line.selectNext();
  },
  type: 'element',
};

// 2. Place AFTER default transformers to avoid conflicts
const EXTENDED_TRANSFORMERS = [...TRANSFORMERS, HR_TRANSFORMER];
```

**RISK ASSESSMENT:** Still risky due to complex interaction patterns in MarkdownShortcuts.ts

## Final Recommendation

**IMPLEMENT OPTION 1 IMMEDIATELY** - Remove HR_TRANSFORMER to restore system stability.

This is a critical system freeze bug that makes the application unusable. The immediate priority must be system stability over feature completeness.

## Immediate Mitigation Recommendations

### Option 1: EMERGENCY FIX - Remove HR_TRANSFORMER (RECOMMENDED)
```typescript
// In src/sidebar/components/LexicalEditor.tsx
// Comment out or remove lines 58-77

// const HR_TRANSFORMER: ElementTransformer = { ... }
// const EXTENDED_TRANSFORMERS = [HR_TRANSFORMER, ...TRANSFORMERS];

// Replace with:
const EXTENDED_TRANSFORMERS = TRANSFORMERS; // Use default transformers only
```

**Pros:** 
- Immediate system stability
- Zero risk of race condition
- Minimal code change

**Cons:**
- Loses horizontal rule markdown shortcuts (user can still use HorizontalRulePlugin)

### Option 2: Use Official Lexical Plugin (SAFER LONG-TERM)
```typescript
// Remove custom HR_TRANSFORMER entirely
// Import official plugin instead:
import { MarkdownShortcutPlugin, DEFAULT_TRANSFORMERS } from '@lexical/react/LexicalMarkdownShortcutPlugin';

// In component JSX:
<MarkdownShortcutPlugin transformers={DEFAULT_TRANSFORMERS} />
```

**Pros:**
- Uses battle-tested official implementation  
- Includes proper transformer ordering
- Maintained by Lexical team

**Cons:**
- Requires dependency check for @lexical/react version compatibility

### 3. Race Condition Execution Path

**Infinite Loop Trigger Sequence:**
1. User types multiple `---` sequences near code blocks
2. MarkdownShortcutPlugin.registerMarkdownShortcuts() is called with EXTENDED_TRANSFORMERS
3. `runElementTransformers()` processes HR_TRANSFORMER first
4. HR regex `/^(---|\*\*\*|___)\s?$/` matches `***` pattern 
5. `parentNode.replace(line)` or `parentNode.insertBefore(line)` executed
6. DOM reconciliation triggers another update cycle
7. **CRITICAL**: Default TRANSFORMERS also have `***` patterns for BOLD_ITALIC formatting
8. Text format transformers attempt to process the same `***` pattern
9. Competing transformers create cascading updates
10. Update listener triggers registerMarkdownShortcuts again  
11. **INFINITE LOOP**: Steps 3-10 repeat indefinitely
12. **SYSTEM FREEZE**: CPU pegged at 100%, system becomes unresponsive

### 4. Key Technical Issues

**Issue 1: Transformer Priority Ordering**
- HR_TRANSFORMER is prepended: `[HR_TRANSFORMER, ...TRANSFORMERS]`
- This places it BEFORE text format transformers that handle `***` and `___`
- Creates race condition between element and text formatters

**Issue 2: Pattern Overlap**
```typescript
// HR_TRANSFORMER matches these patterns:
regExp: /^(---|\*\*\*|___)\s?$/

// But default TRANSFORMERS also include:
BOLD_ITALIC_STAR.tag = '***'      // Conflicts with HR ***
BOLD_ITALIC_UNDERSCORE.tag = '___' // Conflicts with HR ___
```

**Issue 3: Missing Code Block Safety Check**  
The current implementation lacks the crucial safety check:
```typescript
if (parentNode === null || $isCodeNode(parentNode)) {
  return; // Prevent processing inside code blocks
}
```

### 2. Detailed Code Comparison

**Current Implementation (BUGGY):**
```typescript
// src/sidebar/components/LexicalEditor.tsx (lines 58-74)
const HR_TRANSFORMER: ElementTransformer = {
  dependencies: [HorizontalRuleNode],
  export: (node) => {
    return $isHorizontalRuleNode(node) ? '***' : null;
  },
  regExp: /^(---|\*\*\*|___)\s?$/,
  replace: (parentNode, _1, _2, isImport) => {
    const line = $createHorizontalRuleNode();
    if (isImport || parentNode.getNextSibling() != null) {
      parentNode.replace(line);
    } else {
      parentNode.insertBefore(line);
    }
    line.selectNext();
  },
  type: 'element',
};

const EXTENDED_TRANSFORMERS = [HR_TRANSFORMER, ...TRANSFORMERS];
```

**Official Lexical Implementation (WORKING):**
```typescript  
// tmp/lexical-playground/packages/lexical-react/src/LexicalMarkdownShortcutPlugin.tsx
const HR: ElementTransformer = {
  dependencies: [HorizontalRuleNode],
  export: (node: LexicalNode) => {
    return $isHorizontalRuleNode(node) ? '***' : null;
  },
  regExp: /^(---|\*\*\*|___)\s?$/,
  replace: (parentNode, _1, _2, isImport) => {
    const line = $createHorizontalRuleNode();
    
    // TODO: Get rid of isImport flag
    if (isImport || parentNode.getNextSibling() != null) {
      parentNode.replace(line);
    } else {
      parentNode.insertBefore(line);
    }
    
    line.selectNext();
  },
  type: 'element',
};

export const DEFAULT_TRANSFORMERS = [HR, ...TRANSFORMERS];
```

# Critical Race Condition Analysis: Lexical Editor HR_TRANSFORMER

## Problem Summary
- **Critical Issue**: System freeze requiring power cycle
- **Trigger**: Multiple `---` horizontal rules combined with code blocks  
- **Location**: `/home/golem/git/chimera-chat-playground/src/sidebar/components/LexicalEditor.tsx` (lines 58-74)
- **Impact**: Complete system freeze, console inaccessible

## Root Cause Analysis

### 1. Code Architecture Issues

**Problem 1: Conflicting Transformer Processing**
The current implementation uses `EXTENDED_TRANSFORMERS = [HR_TRANSFORMER, ...TRANSFORMERS]` where HR_TRANSFORMER is processed BEFORE the default TRANSFORMERS array. This creates priority conflicts with existing text format transformers.

**Problem 2: Regex Pattern Conflicts**
- HR_TRANSFORMER regex: `/^(---|\*\*\*|___)\s?$/`
- Default TRANSFORMERS include bold/italic patterns for `***` and `___`
- These patterns can overlap and cause recursive processing